In [16]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error
pd.options.display.precision = 15
import tqdm

import lightgbm as lgb
import xgboost as xgb
import time
import datetime

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import Ridge, RidgeCV
import gc
from catboost import CatBoostRegressor
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense
from keras.models import Model

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

#import os
#print(os.listdir("../input"))


ImportError: Traceback (most recent call last):
  File "C:\Users\Shuta\AppData\Local\conda\conda\envs\tf111-gpu\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\Users\Shuta\AppData\Local\conda\conda\envs\tf111-gpu\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "C:\Users\Shuta\AppData\Local\conda\conda\envs\tf111-gpu\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "C:\Users\Shuta\AppData\Local\conda\conda\envs\tf111-gpu\lib\imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "C:\Users\Shuta\AppData\Local\conda\conda\envs\tf111-gpu\lib\imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: DL

In [2]:
#Get data
X_reader = pd.read_csv("lanl-features/train_features.csv", nrows=10000000,
                    dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})

In [5]:
X_reader.head()

,Hann_window_mean_150,Hann_window_mean_1500,Hann_window_mean_15000,Hann_window_mean_50,Hilbert_mean,abs_energy,abs_max,abs_max_roll_mean_10,abs_max_roll_mean_100,abs_max_roll_mean_1000,...,time_rev_asym_stat_1,time_rev_asym_stat_10,time_rev_asym_stat_100,time_rev_asym_stat_1000,time_rev_asym_stat_10000,time_rev_asym_stat_5,time_rev_asym_stat_50,time_rev_asym_stat_500,trend,var_larger_than_std_dev
0,4.883327023570954,4.876660693097298,4.809075953159121,4.883859523420638,7.027028035412704,7481351.0,104.0,68.500000000000000,10.039999999999999,5.629000000000000,...,-0.177655702076028,-14.764688625150020,-1.703217623497997,1.169500000000000,-51.047107692307691,-0.958330555370358,5.087825216811208,5.671832214765103,-0.000003268299817,True
1,4.725049132238183,4.718420573548181,4.653152157320807,4.725527100974546,7.380382529587168,9861777.0,181.0,145.099999999999994,16.670000000000002,5.667000000000002,...,-0.284957132761770,91.001093479130560,-38.729232309746330,-0.665972972972973,-52.383715384615385,71.814107607173810,21.336684456304198,23.323973154362417,0.000000909042448,True
2,4.905511258078014,4.898920772317807,4.832203777557303,4.906040032534657,8.016930170432310,10892549.0,140.0,94.799999999999997,12.380000000000001,5.957000000000002,...,-0.693162575501007,-105.719162555007301,78.751722296395187,4.284682432432431,46.923230769230784,-57.852130142009457,41.037611741160774,46.940375838926172,0.000003962181680,True
3,4.901427864908424,4.894639042521090,4.827671244276916,4.901984819425586,7.606849953983147,10792492.0,199.0,136.599999999999994,18.260000000000002,5.858000000000000,...,0.234849797997307,11.626423523136420,263.667817089452569,20.922364864864864,-13.254976923076926,22.071564770984725,-85.625096731154102,4.832550335570470,0.000001637207225,True
4,4.908115126624049,4.901854958025392,4.836297018529053,4.908528375436501,7.895402664474322,11610208.0,145.0,107.599999999999994,14.410000000000000,6.077999999999999,...,0.126008346777957,12.574096546206160,99.323571428571441,-14.195959459459459,17.525423076923076,18.980185345689712,59.390660440293530,-17.780355704697982,-0.000000666839221,True


In [7]:
y_tr = pd.read_csv("lanl-features/y.csv", nrows=10000000)


In [11]:
#Get data
X_test = pd.read_csv("lanl-features/test_features.csv", 
                    dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})

In [9]:
#特徴量の正規化

scaler = StandardScaler()
scaler.fit(X_reader)
X_train_scaled = pd.DataFrame(scaler.transform(X_reader), columns=X_reader.columns)

C:\Users\Shuta\AppData\Local\conda\conda\envs\tf111-gpu\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Shuta\AppData\Local\conda\conda\envs\tf111-gpu\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  """


In [12]:
#特徴量の正規化

scaler = StandardScaler()
scaler.fit(X_test)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

C:\Users\Shuta\AppData\Local\conda\conda\envs\tf111-gpu\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Shuta\AppData\Local\conda\conda\envs\tf111-gpu\lib\site-packages\ipykernel_launcher.py:5: DataConversionWarning: Data with input dtype bool, int64, float64 were all converted to float64 by StandardScaler.
  """


# ここからLightGBM

In [13]:
model = lgb.LGBMRegressor(n_estimators=10000,learning_rate=0.0005, n_jobs=-1)
model.fit(X_train_scaled,y_tr)
y_pred_lgb = model.predict(X_test_scaled)

# XBboost

In [14]:


model = xgb.XGBRegressor(n_estimators=10)
model.fit(X_train_scaled,y_tr)
y_pred_xgb = model.predict(X_test_scaled)



[00:07:33] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


# NN

In [15]:



inputs = Input(shape=(24,))
x = Dense(128, activation='relu')(inputs)
x = Dense(128, activation='relu')(x)
predictions = Dense(1)(x)
model_3 = Model(inputs=inputs, outputs=predictions)
model_3.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
                      loss='mse',
                    metrics=['accuracy'])
model_3.fit(X_train_scaled, y_tr, epochs=15, verbose=0)
y_pred_nn = model.predict(X_test_scaled).flatten()

ModuleNotFoundError: No module named 'keras'

In [ ]:
type(y_pred_nn)

# 混合

In [ ]:
sample = pd.read_csv("sample_submission.csv")
sample['time_to_failure'] = (y_pred_lgb+y_pred_xgb+y_pred_nn)/3
sample.to_csv('submission.csv',index=False)

# cross varidation


In [ ]:
%time

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss, roc_auc_score

n_estimators = [10,100]

for n_est in  n_estimators:
    print(n_est)
    cv = KFold(n_splits=5, shuffle=True,random_state=0)
    for train, valid in cv.split(X_train_scaled, y_tr):
        x_train = X_train_scaled.iloc[train]
        x_valid = X_train_scaled.iloc[valid]
        y_train = y_tr.iloc[train]
        y_valid = y_tr.iloc[valid]
        
        model_1 = lgb.LGBMRegressor(n_estimators=10000, learning_rate=0.0005,n_jobs=-1,random_state=0)
        model_1.fit(x_train, y_train)
        y_pred_lgb = model_1.predict(x_valid)
        
        model_2 = xgb.XGBRegressor(n_estimators=n_est, n_jobs=-1,random_state=0)
        model_2.fit(x_train, y_train)
        y_pred_xgb = model_2.predict(x_valid)
        
        inputs = Input(shape=(24,))
        x = Dense(128, activation='relu')(inputs)
        x = Dense(128, activation='relu')(x)
        predictions = Dense(1)(x)
        model_3 = Model(inputs=inputs, outputs=predictions)
        model_3.compile(optimizer=tf.train.RMSPropOptimizer(0.001),
                      loss='mse',
                    metrics=['accuracy'])
        model_3.fit(x_train, y_train, epochs=15, verbose=0)
        y_pred_nn = model_3.predict(x_valid).flatten()
        
        y_pred = (y_pred_lgb+y_pred_xgb+y_pred_nn)/3
        print(mean_absolute_error(y_valid, y_pred))  

# ゴミ？

In [ ]:
#訓練データの作成
submission = pd.read_csv('../input/sample_submission.csv', index_col='seg_id')
X_test = pd.DataFrame(columns=X_tr.columns, dtype=np.float64, index=submission.index)
plt.figure(figsize=(22, 16))

for i, seg_id in enumerate(tqdm_notebook(X_test.index)):
    seg = pd.read_csv('../input/test/' + seg_id + '.csv')
    
    x = seg['acoustic_data'].values
    X_test.loc[seg_id, 'ave'] = x.mean()
    X_test.loc[seg_id, 'std'] = x.std()
    X_test.loc[seg_id, 'max'] = x.max()
    X_test.loc[seg_id, 'min'] = x.min()
        
    X_test.loc[seg_id, 'av_change_abs'] = np.mean(np.diff(x))
    X_test.loc[seg_id, 'av_change_rate'] = np.mean(np.nonzero((np.diff(x) / x[:-1]))[0])
    X_test.loc[seg_id, 'abs_max'] = np.abs(x).max()
    X_test.loc[seg_id, 'abs_min'] = np.abs(x).min()
    
    X_test.loc[seg_id, 'std_first_50000'] = x[:50000].std()
    X_test.loc[seg_id, 'std_last_50000'] = x[-50000:].std()
    X_test.loc[seg_id, 'std_first_10000'] = x[:10000].std()
    X_test.loc[seg_id, 'std_last_10000'] = x[-10000:].std()
    
    X_test.loc[seg_id, 'avg_first_50000'] = x[:50000].mean()
    X_test.loc[seg_id, 'avg_last_50000'] = x[-50000:].mean()
    X_test.loc[seg_id, 'avg_first_10000'] = x[:10000].mean()
    X_test.loc[seg_id, 'avg_last_10000'] = x[-10000:].mean()
    
    X_test.loc[seg_id, 'min_first_50000'] = x[:50000].min()
    X_test.loc[seg_id, 'min_last_50000'] = x[-50000:].min()
    X_test.loc[seg_id, 'min_first_10000'] = x[:10000].min()
    X_test.loc[seg_id, 'min_last_10000'] = x[-10000:].min()
    
    X_test.loc[seg_id, 'max_first_50000'] = x[:50000].max()
    X_test.loc[seg_id, 'max_last_50000'] = x[-50000:].max()
    X_test.loc[seg_id, 'max_first_10000'] = x[:10000].max()
    X_test.loc[seg_id, 'max_last_10000'] = x[-10000:].max()
    
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

In [ ]:
#特徴量の作成
%time

rows = 150_000
train = X
segments = int(np.floor(train.shape[0] / rows))

X_tr = pd.DataFrame(index=range(segments), dtype=np.float64,
                       columns=['ave', 'std', 'max', 'min',
                               'av_change_abs', 'av_change_rate', 'abs_max', 'abs_min',
                               'std_first_50000', 'std_last_50000', 'std_first_10000', 'std_last_10000',
                               'avg_first_50000', 'avg_last_50000', 'avg_first_10000', 'avg_last_10000',
                               'min_first_50000', 'min_last_50000', 'min_first_10000', 'min_last_10000',
                               'max_first_50000', 'max_last_50000', 'max_first_10000', 'max_last_10000'])
y_tr = pd.DataFrame(index=range(segments), dtype=np.float64,
                       columns=['time_to_failure'])

total_mean = train['acoustic_data'].mean()
total_std = train['acoustic_data'].std()
total_max = train['acoustic_data'].max()
total_min = train['acoustic_data'].min()
total_sum = train['acoustic_data'].sum()
total_abs_max = np.abs(train['acoustic_data']).sum()

for segment in tqdm_notebook(range(segments)):
    seg = train.iloc[segment*rows:segment*rows+rows]
    x = seg['acoustic_data'].values
    y = seg['time_to_failure'].values[-1]
    
    y_tr.loc[segment, 'time_to_failure'] = y
    X_tr.loc[segment, 'ave'] = x.mean()
    X_tr.loc[segment, 'std'] = x.std()
    X_tr.loc[segment, 'max'] = x.max()
    X_tr.loc[segment, 'min'] = x.min()
    
    
    X_tr.loc[segment, 'av_change_abs'] = np.mean(np.diff(x))
    X_tr.loc[segment, 'av_change_rate'] = np.mean(np.nonzero((np.diff(x) / x[:-1]))[0])
    X_tr.loc[segment, 'abs_max'] = np.abs(x).max()
    X_tr.loc[segment, 'abs_min'] = np.abs(x).min()
    
    X_tr.loc[segment, 'std_first_50000'] = x[:50000].std()
    X_tr.loc[segment, 'std_last_50000'] = x[-50000:].std()
    X_tr.loc[segment, 'std_first_10000'] = x[:10000].std()
    X_tr.loc[segment, 'std_last_10000'] = x[-10000:].std()
    
    X_tr.loc[segment, 'avg_first_50000'] = x[:50000].mean()
    X_tr.loc[segment, 'avg_last_50000'] = x[-50000:].mean()
    X_tr.loc[segment, 'avg_first_10000'] = x[:10000].mean()
    X_tr.loc[segment, 'avg_last_10000'] = x[-10000:].mean()
    
    X_tr.loc[segment, 'min_first_50000'] = x[:50000].min()
    X_tr.loc[segment, 'min_last_50000'] = x[-50000:].min()
    X_tr.loc[segment, 'min_first_10000'] = x[:10000].min()
    X_tr.loc[segment, 'min_last_10000'] = x[-10000:].min()
    
    X_tr.loc[segment, 'max_first_50000'] = x[:50000].max()
    X_tr.loc[segment, 'max_last_50000'] = x[-50000:].max()
    X_tr.loc[segment, 'max_first_10000'] = x[:10000].max()
    X_tr.loc[segment, 'max_last_10000'] = x[-10000:].max()